In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/nudt/.julia/packages/Plots/kyYZF/src/backends.jl:372


In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

1

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=-0.15
η2=0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=20/3*ν
γ2=40/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);


In [5]:
function sweep_detuning_200()
    ttot=200
    initial_x=[5.0 16]
    sweep_num=120
    start=30
    stop=150
    d=[]
    somega_g=[]
    somega_r=[]
    sn=[]
    for i in 1:sweep_num
        println(" ")
        println(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],m))*instate
            nbose=real(tr(n*vec2dm(f)))
            print(" f ",nbose," ")
            return nbose
        end        

        a=optimize(feval,initial_x,LBFGS())
        initial_x=Optim.minimizer(a) 
        append!(somega_g,initial_x[1])  
        append!(somega_r,initial_x[2])
        append!(sn,Optim.minimum(a)[1])
        append!(d,m)
        df1=DataFrame(delta=d,omega_g=somega_g,omega_r=somega_r,n=sn);CSV.write("eit_sweep_detuning_200.csv", df1);
    end
    return somega_g,somega_r,sn,d
end
@time res2=sweep_detuning_200()


 
1 
 f 0.4574237263409401  f 0.4574250796603845  f 0.4574362472122886  f 0.4574125589232924  f 0.4574244029829117  f 0.44212034007383916  f 0.4421216708867151  f 0.442132543207121  f 0.4421094679009066  f 0.44212100546207744  f 0.38404615021818533  f 0.38404736641078496  f 0.38405698352251216  f 0.38403653325808207  f 0.3840467582957127  f 0.16557687584819825  f 0.1655760110422398  f 0.16558155111048092  f 0.1655713359193285  f 0.16557644343815037  f 0.9851939173370482  f 0.9851973963215681  f 0.9851947015025124  f 0.985196612185935  f 0.9851956568480302  f 0.08576333728120487  f 0.08576347190241707  f 0.08576312539971095  f 0.08576368382908195  f 0.085763404588157  f 0.08519079352202934  f 0.08519064325661871  f 0.08519073471822032  f 0.08519070210600059  f 0.0851907183860609  f 0.08517166593468944  f 0.08517155627410042  f 0.08517158391268695  f 0.08517163834158271  f 0.08517161110099827  f 0.08506412563672489  f 0.08506399915973939  f 0.08506402970254673  f 0.08506409514094987  f 0

 f 0.06473655987159072  f 0.06473664482656791  f 0.06473621090258279  f 0.06473699394643383  f 0.06473660234443411  f 0.06470146467547505  f 0.06470154546845105  f 0.06470112799034103  f 0.06470188230439075  f 0.06470150506736187  f 0.06457472780156445  f 0.06457479232814922  f 0.06457444020969998  f 0.06457508007063674  f 0.06457476006038575  f 0.0642598031328186  f 0.06425979489858642  f 0.06425976116830835  f 0.06425983701434922  f 0.06425979901154008  f 0.07009834471704544  f 0.07009814453849739  f 0.07009967876790052  f 0.07009681068558343  f 0.0700982446238677  f 0.06425678367674921  f 0.0642567680121599  f 0.06425676981016505  f 0.06425678203023978  f 0.06425677584028133  f 0.06425410611444364  f 0.06425409579379801  f 0.06425409706020678  f 0.06425410500061972  f 0.06425410095002664  f 0.06425465922047477  f 0.06425467094882381  f 0.06425467028819477  f 0.0642546600370564  f 0.06425466508035213  f 0.06425208430308967  f 0.0642520840644979  f 0.06425208439736406  f 0.06425208412

 f 0.05777013550670346  f 0.057770215422667276  f 0.057769793685848746  f 0.05777055741175748  f 0.05777017546046016  f 0.05774096767200044  f 0.057741044018019064  f 0.05774063756346481  f 0.05774137429480638  f 0.057741005840814645  f 0.05763513076402407  f 0.057635193140622404  f 0.057634847465659576  f 0.05763547660707573  f 0.05763516194819494  f 0.057359860263059274  f 0.05735985981190159  f 0.05735981111377539  f 0.05735990913006842  f 0.057359860033624636  f 0.061951218744573815  f 0.06195104696582088  f 0.061952470206239146  f 0.06194979572186  f 0.06195113285172181  f 0.057354183502278576  f 0.05735417332301217  f 0.05735417496319491  f 0.05735418203138123  f 0.05735417840882806  f 0.05735307407392298  f 0.05735306708768582  f 0.05735306785634126  f 0.05735307347505199  f 0.0573530705769375  f 0.05735270949229881  f 0.05735271550368406  f 0.057352712837637945  f 0.057352712330699  f 0.05735271249409856  f 0.057352058056182785  f 0.05735205818521972  f 0.05735205705151157  f 0

 f 0.05452197908856212  f 0.05452205594360363  f 0.054521639850071614  f 0.05452239536551091  f 0.054522017512128595  f 0.054496097774022104  f 0.05449617138687762  f 0.05449577001695944  f 0.05449649932727814  f 0.054496134576497456  f 0.054401953998668776  f 0.05440201491185436  f 0.054401672136144975  f 0.05440229695757042  f 0.05440198445135503  f 0.054151655459139345  f 0.05415165898558731  f 0.05415160326836576  f 0.05415171136056673  f 0.05415165721874486  f 0.05812442682323849  f 0.05812426926349492  f 0.05812564429723681  f 0.058123052024227045  f 0.058124348040070516  f 0.05414475978101758  f 0.05414475287434902  f 0.05414475427044075  f 0.05414475856964907  f 0.054144756324080084  f 0.054144263390640573  f 0.05414425847522411  f 0.05414425900229406  f 0.054144263048682834  f 0.054144260929364536  f 0.05414389129506644  f 0.05414389442878492  f 0.05414389149038427  f 0.05414389442027959  f 0.05414389285828815  f 0.05414372558448778  f 0.05414372580109654  f 0.0541437241099886

 f 0.05245414515395566  f 0.05245421911997946  f 0.05245380633873991  f 0.052454558138812224  f 0.052454182133254174  f 0.05243122663491106  f 0.052431297642409204  f 0.05243089920687784  f 0.05243162527398401  f 0.052431262135008386  f 0.05234770560290148  f 0.05234776500733392  f 0.05234742370603505  f 0.052348047107614905  f 0.05234773530145712  f 0.052122236735709745  f 0.05212224339273786  f 0.05212218291417574  f 0.05212229741902609  f 0.05212224006081124  f 0.05559386426082633  f 0.05559371719112071  f 0.05559506584473504  f 0.05559251586456816  f 0.05559379072292448  f 0.05211483274775188  f 0.05211482882555525  f 0.05211482985504238  f 0.05211483192384856  f 0.05211483078334768  f 0.05211468045418728  f 0.052114677541594684  f 0.052114677881996266  f 0.05211468031954382  f 0.0521146789945561  f 0.05211449583433486  f 0.05211449698056146  f 0.05211449456313364  f 0.052114498458576256  f 0.05211449640411291  f 0.05211447788968021  f 0.05211447808683062  f 0.05211447631249053  f 

...

 f 0.12266147979591743  f 0.12266155513213416  f 0.1226610662439319  f 0.12266196935825048  f 0.12266151746054241  f 0.1226496881540388  f 0.1226497603348715  f 0.12264929113089199  f 0.1226501580324513  f 0.12264972424115356  f 0.12260737617616954  f 0.12260743584165573  f 0.12260704533944015  f 0.12260776735408244  f 0.12260740600561923  f 0.12251193464049275  f 0.12251193422174426  f 0.12251193663904228  f 0.12251193290514101  f 0.12251193442808501  f 0.1225119318157162  f 0.12251193170155637  f 0.12251193205706067  f 0.12251193214186805  f 0.12251193175540787  f 0.12251193176606492  f 0.12251193165618474  f 0.12251193200661226  f 0.12251193209730683  f 0.12251193170806954  f 0.12251193158246919  f 0.12251193148895134  f 0.12251193181952147  f 0.12251193193359262  f 0.12251193153259814  f 0.12251193101911446  f 0.12251193100736316  f 0.12251193123856641  f 0.12251193146978534  f 0.12251193101010398  f 0.122511937079128  f 0.12251193747719223  f 0.122511937212939  f 0.122511938025948

 f 0.12878922238967788  f 0.12878929780899961  f 0.12878880405075724  f 0.12878971685528506  f 0.12878926009589217  f 0.1287776259568629  f 0.12877769818404253  f 0.12877722450960982  f 0.12877810033922207  f 0.12877766206705132  f 0.12873606353328057  f 0.12873612309748897  f 0.128735729720423  f 0.12873645761919647  f 0.128736093312033  f 0.1286436218698632  f 0.128643620606567  f 0.1286436280778665  f 0.12864361511318306  f 0.12864362123498116  f 0.12864359130088754  f 0.12864359107684187  f 0.12864359147286258  f 0.12864359161954445  f 0.1286435911857258  f 0.12864359112209472  f 0.1286435909067609  f 0.12864359128344183  f 0.12864359146046667  f 0.12864359101132813  f 0.1286435904594849  f 0.12864359027942804  f 0.12864359057758995  f 0.12864359087614735  f 0.1286435903663096  f 0.12864358840739404  f 0.12864358840419843  f 0.1286435883103831  f 0.12864358921635596  f 0.12864358840257428  f 0.1286436097136987  f 0.12864361059479962  f 0.12864360854368928  f 0.12864361248151654  f 

 f 0.13494456017415868  f 0.1349446355542127  f 0.13494413672348068  f 0.1349450597463087  f 0.1349445978606891  f 0.13493313377226107  f 0.1349332059261863  f 0.13493272759989783  f 0.13493361284037794  f 0.1349331698457502  f 0.13489223128515632  f 0.134892290640207  f 0.13489189429191187  f 0.13489262837609853  f 0.13489226095923137  f 0.13480260339599678  f 0.13480260125552418  f 0.13480261409156724  f 0.13480259130881606  f 0.13480260232255023  f 0.13480251430829268  f 0.13480251397217838  f 0.1348025144114611  f 0.13480251461803583  f 0.13480251413696545  f 0.13480251392513212  f 0.1348025136025311  f 0.13480251400582413  f 0.13480251427090081  f 0.13480251376055968  f 0.13480251250092012  f 0.13480251223142828  f 0.1348025124906654  f 0.13480251299060056  f 0.1348025123629687  f 0.13480250796339327  f 0.13480250795894116  f 0.1348025074991955  f 0.13480250917229908  f 0.13480250795799525  f 0.1348025499465454  f 0.13480255126766993  f 0.1348025472169476  f 0.13480255474856398  f

 f 0.1411127856338695  f 0.14111286083675745  f 0.14111235647587644  f 0.14111329077233167  f 0.14111282323191565  f 0.14110149489937715  f 0.1411015668432107  f 0.1411010834512386  f 0.14110197906913616  f 0.1411015308678455  f 0.14106113271866372  f 0.14106119173413412  f 0.14106079217475073  f 0.14106153305686356  f 0.14106116222300558  f 0.14097414365674063  f 0.140974140559618  f 0.1409741593718508  f 0.14097412562962044  f 0.1409741421051388  f 0.140973958058675  f 0.1409739575980634  f 0.1409739580849835  f 0.14097395835654544  f 0.14097395782532868  f 0.14097395736797913  f 0.14097395692478806  f 0.14097395735710994  f 0.14097395772019697  f 0.14097395714315064  f 0.14097395478518882  f 0.1409739544122923  f 0.14097395462655388  f 0.14097395535611001  f 0.1409739545956084  f 0.14097394620896958  f 0.14097394618749545  f 0.14097394531011867  f 0.14097394787141065  f 0.1409739461950009  f 0.14097401194618114  f 0.14097401367925894  f 0.14097400735116508  f 0.14097401906261747  f 

(Any[3.7482402468121934, 3.7692416973803464, 3.7915139494932237, 3.814945146728345, 3.8394401240649554, 3.8649153378293266, 3.8912973495156216, 3.918521171299918, 3.946527730236973, 3.9752636227692255  …  7.080238289208542, 7.09861105724911, 7.1165738185707275, 7.13411102384809, 7.1512054079407985, 7.167835653866557, 7.183969533476006, 7.199578722316161, 7.214633715548907, 7.229093277583029], Any[10.853925205453864, 11.024551115861392, 11.192703918680767, 11.35847660154247, 11.521954897548195, 11.683218375693862, 11.842341678517235, 11.999394783536761, 12.15444381179963, 12.307551385889976  …  22.91737665476875, 22.999275896742134, 23.08097644688807, 23.162485310025613, 23.243809675763764, 23.324957833574402, 23.405940237864467, 23.486766871926022, 23.56744745926237, 23.647995131897503], Any[0.07327112000770682, 0.07063487763890049, 0.0682718942784309, 0.0661527399516954, 0.06425208333536878, 0.06254802549871762, 0.06102155596788575, 0.05965610623407141, 0.058437181339809664, 0.0573520